# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.interpret import ExplanationClient
from azureml.pipeline.steps import AutoMLStep


In [2]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


### Workspace 

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="automl-udacity-capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135035
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-135035


### Compute Cluster

In [4]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

ws = Workspace.from_config() # this automatically looks for a directory .azureml

#Create compute cluster
cluster_name="automl-project"

try:
  compute_target=ComputeTarget(workspace=ws,name=cluster_name)
  print("Found existing compute target")
except ComputeTargetException:
  print('Creating a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                       max_nodes=5, 
                                                       idle_seconds_before_scaledown=2400,
                                                       vm_priority='lowpriority')
  compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
  

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes =  2)

Creating a new compute target
Creating
Succeeded....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

Capstone-udacity ComputeInstance Succeeded
hd-project AmlCompute Succeeded
automl-project AmlCompute Succeeded


## Dataset

### Overview

**Citation Source** :

Creator:

B. German
Central Research Establishment
Home Office Forensic Science Service
Aldermaston, Reading, Berkshire RG7 4PN

Donor:

Vina Spiehler, Ph.D., DABFT
Diagnostic Products Corporation
(213) 776-0180 (ext 3014)


**Data Set Information**:

Vina conducted a comparison test of her rule-based system, BEAGLE, the nearest-neighbor algorithm, and discriminant analysis. BEAGLE is a product available through VRS Consulting, Inc.; 4676 Admiralty Way, Suite 206; Marina Del Ray, CA 90292 (213) 827-7890 and FAX: -3189. In determining whether the glass was a type of "float" glass or not, the following results were obtained (# incorrect answers):

Type of Sample -- Beagle -- NN -- DA
Windows that were float processed (87) -- 10 -- 12 -- 21
Windows that were not: (76) -- 19 -- 16 -- 22

The study of classification of types of glass was motivated by criminological investigation. At the scene of the crime, the glass left can be used as evidence...if it is correctly identified!


Attribute Information:

1. Id number: 1 to 214
2. RI: refractive index
3. Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
4. Mg: Magnesium
5. Al: Aluminum
6. Si: Silicon
7. K: Potassium
8. Ca: Calcium
9. Ba: Barium
10. Fe: Iron
11. Type of glass: (class attribute)
- 1 building_windows_float_processed
- 2 building_windows_non_float_processed
- 3 vehicle_windows_float_processed
- 4 vehicle_windows_non_float_processed (none in this database)
- 5 containers
- 6 tableware
- 7 headlamps


Relevant Papers:

Ian W. Evett and Ernest J. Spiehler. Rule Induction in Forensic Science. Central Research Establishment. Home Office Forensic Science Service. Aldermaston, Reading, Berkshire RG7 4PN
[Web Link]


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
# Import dataset
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType

# create a TabularDataset from a delimited file behind a public web url 
web_path ="https://raw.githubusercontent.com/Aishwaryasasanapuri/test2/main/glass.csv"
final_df = Dataset.Tabular.from_delimited_files(path=web_path) 

# preview the first 3 rows of titanic_ds
final_df.take(3).to_pandas_dataframe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 10,
    "max_concurrent_iterations": 8,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data = final_df,
                             label_column_name = 'Type',
                             iterations= 100,
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
automl_run = exp.submit(automl_config)

Running on remote.


In [9]:
automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-udacity-capstone,AutoML_d978cdf9-f9e8-4561-b163-c40905bf025c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(run).show()

# Wait for the remote run to complete
automl_run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+==========================

{'runId': 'AutoML_d978cdf9-f9e8-4561-b163-c40905bf025c',
 'target': 'automl-project',
 'status': 'Completed',
 'startTimeUtc': '2021-01-17T11:33:33.57527Z',
 'endTimeUtc': '2021-01-17T11:56:27.435771Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl-project',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-udacity-capstone","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-135035","workspace_name":"quick-starts-ws-135035","region":"southcentralus","compute_target":"automl-project","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":nu

In [11]:
automl_run.get_metrics('accuracy')

{'accuracy': 0.7566998892580289}

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.


Retrieve the Best Model's explanation from the best_run which includes explanations for engineered features and raw features. Make sure that the run for generating explanations for the best model is completed.

In [14]:
# Wait for the best model explanation run to complete
from azureml.core.run import Run
model_explainability_run_id = automl_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=exp, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

# Get the best run object
automl_best_run, model = automl_run.get_output()
automl_best_run_metrics = automl_best_run.get_metrics()

AutoML_d978cdf9-f9e8-4561-b163-c40905bf025c_ModelExplain


In [15]:
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-udacity-capstone,AutoML_d978cdf9-f9e8-4561-b163-c40905bf025c_44,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                            ('kneighborsclassifier',
                                                                             KNeighborsClassifier(algorithm='auto',
                                                                                                  leaf_size=30,
                                                               

In [17]:
print('Best Run Id: ', automl_best_run.id)
print('\n Accuracy:', automl_best_run_metrics['accuracy'])
print('\n AUC Weighted:', automl_best_run_metrics['AUC_weighted'])

Best Run Id:  AutoML_d978cdf9-f9e8-4561-b163-c40905bf025c_44

 Accuracy: 0.7566998892580289

 AUC Weighted: 0.9421288508160044


#### Describing the Best model

In [18]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['3', '4', '6', '36', '22', '29', '9'],
 'weights': [0.2222222222222222,
             0.1111111111111111,
             0.1111111111111111,
             0.1111111111111111,
             0.1111111111111111,
             0.2222222222222222,
             0.1111111111111111]}

3 - robustscaler
{'copy': True,
 'quantile_range': [10, 90],
 'with_centering': False,
 'with_scaling': False}

3 - extratreesclassifier
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min

In [19]:
model_name = automl_best_run.properties['model_name']
model_name

'AutoMLd978cdf9f44'

In [20]:
#TODO: Save the best model
import joblib

os.makedirs('./outputs', exist_ok=True)
#best_model_automl=

joblib.dump(model,filename='outputs/best_automlmodel.pkl')

['outputs/best_automlmodel.pkl']

In [21]:
automl_best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":5}',
 'ensembled_iterations': '[3, 4, 6, 36, 22, 29, 9]',
 'ensembled_algorithms': "['ExtremeRandomTrees', 'RandomForest', 'RandomForest', 'XGBoostClassifier', 'ExtremeRandomTrees', 'SVM', 'KNN']",
 'ensemble_weights': '[0.2222222222222222, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.2222222222222222, 0.1111111111111111]',
 'best_individual_pipeline_score': '0.9281389988761616',
 'best_individual_iteration': '3',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_d978cdf9-f9e8-4561-b163-c40905bf025c_ModelExplain',
 'model_explanation': 'True'}

## Registering Model

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ='outputs/best_automlmodel.pkl',
                        model_name = "automlmodel")

Registering model automlmodel


In [23]:
print(model)
print(model.id)

Model(workspace=Workspace.create(name='quick-starts-ws-135035', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-135035'), name=automlmodel, id=automlmodel:1, version=1, tags={}, properties={})
automlmodel:1


TODO: In the cell below, send a request to the web service you deployed to test it.

#### Model Deployment

### Setting up Inference config and ACI config

In [24]:
#env = automl_best_run.get_environment().save_to_directory(path='environments')

script_file_name = 'score.py'

automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

In [25]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies


#inference_config = InferenceConfig(entry_script='Score.py',source_directory='.',environment=myenv)
inference_config = InferenceConfig(entry_script=script_file_name,environment=automl_best_run.get_environment())  #,source_directory='.'

#inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1,
                                               enable_app_insights=True, 
                                               auth_enabled=True, 
                                               description = 'Service for Automl Classification')

In [26]:
#aci_service_name = 'automl-capstone'

aci_service_name = 'amlcapstone'

print(aci_service_name)

amlcapstone


In [27]:
aci_service = Model.deploy(ws,aci_service_name,  [model], inference_config, aciconfig)

aci_service.wait_for_deployment(show_output = True)

print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 4b3a27bb-1205-45e7-aba0-3a25a76112f8
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: amlcapstone. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_4f3cee89203e005745d1830c04fe722a locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 4b3a27bb-1205-45e7-aba0-3a25a76112f8
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: amlcapstone. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_4f3cee89203e005745d1830c04fe722a locally. Please refer to https://aka.ms/debugimage#service-launch-fai

In [ ]:
print(aci_service.get_logs())

In [ ]:
print("scoring URI: " + aci_service.scoring_uri)
print("Swagger URI: " + aci_service.swagger_uri)
print("Authetication Key: " + aci_service.get_keys()[0])

#### Since we are facing issue deploying the ACI service , I have deployed the service manually through Portal

#### Call the services

In [28]:
import requests
import json

# URL for the web service
scoring_uri = 'http://aea0d434-7b93-4641-af8e-9240aafd1c0b.southcentralus.azurecontainer.io/score'
# aci_service.swagger_uri #'<your web service URI>'

# If the service is authenticated, set the key or token
key =  'mP2nsRiJnjhOxCK7vZYgORXbpA3HFXyQ'
#aci_service.get_keys()[0]  #'<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            {
               'RI': 5,
               'Na': 2,
               'Mg': 3.5,
               'Al': 0.7,
               'Si': 83,
               'K': 1.2,
               'Ca': 9,
               'Ba': 0.12,
               'Fe': 0.15
               
               },
            {
               'RI': 2,
               'Na': 25,
               'Mg': 2.5,
               'Al': 1.7,
               'Si': 63,
               'K': 2.2,
               'Ca': 6,
               'Ba': 0.15,
               'Fe': 0.18   
           }
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

"{\"result\": [1, 5]}"


### Testing the deployed service

In [29]:
%run endpoint.py

"{\"result\": [1, 5]}"


### Delete a Web Service

TODO: In the cell below, print the logs of the web service and delete the service

In [33]:
aci_service.delete()

#### Deleting compute

In [ ]:
compute_target.delete()